In [1]:
# The required packages
using Printf
using Plots
using ProgressBars


To download packages required run the following cells:


In [2]:
using Pkg
Pkg.add("Plots")
Pkg.add("ProgressBars")


    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


lets define the constants

In [3]:
const L       = 50             # Define size of lattice
const n_sweep = 20             # Define number of sweeps between sampling
const n_therm = 100       # Define number of sweeps 
const n_data  = 10000            # Define number of data samples per temperature
const temps   = 4.0:-0.3:0.1   # Define temperatures range 


4.0:-0.3:0.1

Define a function that measures the i'th sample of energy and magnetization

In [4]:
function measure(i, energy, magnetization, s)      
    en = 0
    m = 0
    for x = 1:L
        for y = 1:L
            u = 1+mod(y,L) # up 
            r = 1+mod(x,L) # right 
            en -= s[x,y]*(s[x,u]+s[r,y]) # energy
            m  += s[x,y]                 # magnetization
        end
    end
    energy[i] = en
    magnetization[i] = abs(m)
end

measure (generic function with 1 method)

implementing flip function to apply metropolis spin flip algorithm to site (x,y) / temp T

In [5]:
function flip(x, y, T, s)
    u = 1+mod(y,L)   # up
    d = 1+mod(y-2,L) # down
    r = 1+mod(x,L)   # right
    l = 1+mod(x-2,L) # left
    de = 2*s[x,y]*(s[x,u]+s[x,d]+s[l,y]+s[r,y])
    if (de < 0)
        s[x,y] = -s[x,y]
    else
        p = rand()
        if (p < exp(-de/T))
            s[x,y] = -s[x,y]
        end
    end
end


flip (generic function with 1 method)

Implementing sweeb function to apply  flip function to every site on the lattice

In [6]:
function sweep(n, T, s) 
    for i = 1:n
        for x = 1:L
            for y = 1:L
                flip(x,y,T, s)
            end
        end
    end
end

sweep (generic function with 1 method)

Implement main julia function for algorithm opreating and calculation of physical quantities

Now lets plot the physical quantities vs temperature

In [13]:
function main()
    e1 = Array(1:n_data)     # This array purpose is to hold energy measurements (fixed T)
    m1 = Array(1:n_data)     # This array purpose is to hold magnetization measurements (fixed T)
    en = []                  # This array purpose is to append average energy at each T
    mz = []                  # This is magnetization array
    s  = ones(Int32,L,L)     # lattice of Ising spins (+/-1)
    for T in tqdm(temps)              # Lets loop over temperatures range
        sweep(n_therm, T, s)    # Sweebs over the lattice to assign tempreatures
        energy        = e1      # Reset energy measurement array
        magnetization = m1      # Reset magnetization measurement array
        for i = 1:n_data        # Take n_data measurements w/ n_sweep 
            sweep(n_sweep, T, s)   
            measure(i, energy, magnetization, s)
        end
        avg_en = sum(energy)/n_data           # calculate average energy
        avg_ma = sum(magnetization)/n_data    # calculate average magnetization
        push!(en,avg_en/(L*L))                # Add those to the list
        push!(mz,avg_ma/(L*L))
        # @printf("%8.3f  %8.3f \n", avg_en/(L*L), avg_ma/(L*L)) # using this for debugging code purposes to see values.
    end
    gr() # Set the backend to GR
    plot(temps,mz,title = "Magnetization vs Temperature",xlabel = "Temperature",ylabel = "Magnetization",label = "Magnetization") # plot Magnetization vs Temperature
    savefig("plots/Magnetization.pdf")
    plot(temps,en,title = "Energy vs Temperature",xlabel = "Temperature",ylabel = "Energy",label="Energy") # plot Energy vs. Temperature
    savefig("plots/Energy.pdf")
end

main()

0.0%┣                                               ┫ 0/14 [00:00<00:-1, -0s/it]
7.1%┣███                                        ┫ 1/14 [00:21<Inf:Inf, InfGs/it]
14.3%┣██████▋                                       ┫ 2/14 [00:42<08:27, 42s/it]
21.4%┣█████████▉                                    ┫ 3/14 [01:03<05:46, 31s/it]
28.6%┣█████████████▏                                ┫ 4/14 [01:23<04:38, 28s/it]
35.7%┣████████████████▍                             ┫ 5/14 [01:43<03:52, 26s/it]
42.9%┣███████████████████▊                          ┫ 6/14 [02:02<03:15, 24s/it]
50.0%┣███████████████████████                       ┫ 7/14 [02:18<02:41, 23s/it]
57.1%┣██████████████████████████▎                   ┫ 8/14 [02:33<02:11, 22s/it]
64.3%┣█████████████████████████████▋                ┫ 9/14 [02:48<01:45, 21s/it]
71.4%┣████████████████████████████████▏            ┫ 10/14 [03:02<01:21, 20s/it]
78.6%┣███████████████████████████████████▍         ┫ 11/14 [03:17<00:59, 20s/it]
85.7%┣██████████████████████